###### <img align="left" src="./images/movie_camera.png"     style=" width:40px;  " > Collaborative Filtering Recommender Systems

 Objective : Implement collaborative filtering on the MovieLens dataset to predict user ratings for movies, enhancing recommendation accuracy and user satisfaction, culminating in a functional recommender system for personalized movie suggestions.


##  Packages 


In [64]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import precision_score, recall_score

## 1 - Notation used 


1. $r(i,j)$: Binary indicator that is set to 1 if user $j$ rated movie $i$.  
   
2. $y(i,j)$: Rating given by user $j$ for movie $i$  

3. $\mathbf{w}^{(j)}$: Parameter vector for user $j$.  

4. $b^{(j)}$: Bias parameter for user $j$.  

5. $\mathbf{x}^{(i)}$: Feature vector for movie $i$.  

6. $n_u$:   Number of users.  

7. $n_m$: Number of movies.  

8. $n$: Number of features.  

## 2 - Collaborative Filtering 


 **Objective**:
   - The goal is to recommend movies to users based on their preferences. For this purpose, two vectors are required.

 **Vectors**:
   - Two types of vectors are involved:
     - Parameter vectors for users: Represent users' movie tastes.
     - Feature vectors for movies: Describe movie characteristics.
   - The next question is, after obtaining parameters, how do you predict?

 **Dot Product**:
   - Taking the dot product of a user's parameter vector and a movie's feature vector, along with a bias term, helps estimate how much a user might like a movie.

 **Estimation**:
   - This estimation provides a prediction of the rating a user might give to a particular movie.

 **Personalization**:
   - Collaborative filtering allows for personalized movie recommendations by understanding both user preferences and movie attributes.



<figure>
   <img src="./images/ColabFilter.PNG"  style="width:740px;height:350px;" >
</figure>

 **User-Item Interface Matrix**: 
  - The matrix contains ratings given by users to movies. 
  - Users are represented by rows, and movies are represented by columns.
  - Ratings range from 0 to 10, with -1 indicating that the movie has not been rated by the user.



 **Parameter Vectors for Users ($\mathbf{w}^{\text{user}}$) and Bias**:
  - Each user has associated parameter vectors and a bias term.
  - These parameters represent the preferences and tendencies of each user towards different movie features.
  - The parameters are learned based on the data in each row of the user-item interface matrix.

 **Feature Vectors for Movies ($\mathbf{x}^{\text{movie}}$)**:
  - Each movie has associated feature vectors.
  - These vectors describe the characteristics or attributes of each movie, such as genre, cast, director, etc.
  - The feature vectors are learned based on the data in each column of the user-item interface matrix.
  
 **Simultaneous Learning**:
  - The parameter vectors for users and the feature vectors for movies are learned simultaneously using the existing user-movie ratings as training data.
  - By analyzing the patterns in the user-item interface matrix, the model adjusts the parameter and feature vectors to better predict user ratings.

In summary, the user-item interface matrix contains user ratings for movies, and based on this data, the model learns parameter vectors for users and feature vectors for movies. These vectors capture user preferences and movie characteristics, respectively, and are adjusted simultaneously during the learning process to improve the accuracy of rating predictions.

**Prediction**:

Once the feature vectors and parameters are learned, they're used to predict how a user might rate an unrated movie. 
For example, in the image  user-1 didn't rated movie 2, the prediction is calculated by multiplying their preferences ($\mathbf{w}^{\text{user}}_1$) with the movie's features ($\mathbf{x}^{\text{movie}}_2$), and adding their bias term ($b^{\text{user}}_1$):

$$
\text{Prediction}_{\text{user-1, movie-2}} = \mathbf{w}^{\text{user}}_1 \cdot \mathbf{x}^{\text{movie}}_2 + b^{\text{user}}_1
$$

This gives an estimate of how user-1 might rate movie 2, based on their preferences and the movie's characteristics.


<a name="3"></a>
## 3 - Movie ratings dataset 
The data set is derived from the [MovieLens "ml-latest-small"](https://grouplens.org/datasets/movielens/latest/) dataset.   

Detailed description of the dataset and the matrices involved in the collaborative filtering algorithm:

- The dataset contains ratings for 9000 movies by 600 users,The dataset has been reduced in size to focus on movies from the years since 2000.
- Ratings range from 0.5 to 5 in 0.5 increments.
- The reduced dataset has 443 users and 4778 movies.
- Matrices $Y$ ($n_m \times n_u$ matrix) and $R$ store the ratings and indicators, respectively.
- Matrix $\mathbf{X}$ contains feature vectors for movies, $\mathbf{W}$ contains parameter vectors for users, and $\mathbf{b}$ contains bias terms.
$$\mathbf{X} = 
\begin{bmatrix}
(\mathbf{x}^{(0)})^T  \\
(\mathbf{x}^{(1)})^T  \\
\vdots \\
(\mathbf{x}^{(n_m-1)})^T \\
\end{bmatrix} , \quad
\mathbf{W} = 
\begin{bmatrix}
(\mathbf{w}^{(0)})^T \\
(\mathbf{w}^{(1)})^T \\
\vdots \\
(\mathbf{w}^{(n_u-1)})^T \\
\end{bmatrix},\quad
\mathbf{ b} = 
\begin{bmatrix}
 b^{(0)}  \\
 b^{(1)} \\
\vdots \\
b^{(n_u-1)} \\
\end{bmatrix}\quad
$$ 
- Each feature vector $\mathbf{x}^{(i)}$ and parameter vector $\mathbf{w}^{(j)}$ has 10 elements.
- $\mathbf{X}$ is a $n_m \times 10$ matrix, and $\mathbf{W}$ is a $n_u \times 10$ matrix.
- The $i$-th row of $\mathbf{X}$ corresponds to the
feature vector $x^{(i)}$ for the $i$-th movie, and the $j$-th row of
$\mathbf{W}$ corresponds to one parameter vector $\mathbf{w}^{(j)}$, for the
$j$-th user.

In [53]:
file = open('./data/Y.csv', 'rb')
Y = loadtxt(file,delimiter = ",")

file = open('./data/R.csv', 'rb')
R = loadtxt(file,delimiter = ",")

print("Y", Y.shape, "R", R.shape)

Y (4778, 443) R (4778, 443)


In [54]:
#  From the matrix, we can compute statistics like average rating.
tsmean =  np.mean(Y[0, R[0, :].astype(bool)])
print(f"Average rating for movie 1 : {tsmean:0.3f} / 5" )

Average rating for movie 1 : 3.400 / 5


## 4 - Collaborative filtering learning algorithm 

The objective is to implement the collaborative filtering learning algorithm by defining the objective function. In this algorithm, we have parameter vectors $\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)}$, $\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$, and bias terms $b^{(0)},...,b^{(n_u-1)}$. 

The model predicts the rating for movie $i$ by user $j$ as:

$$
y^{(i,j)} = \mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)}
$$

Given a dataset of ratings produced by users on movies, we aim to learn the parameter vectors $\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)}$, $\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$, and $b^{(0)},...,b^{(n_u-1)}$ that minimize the squared error.

### 4.1 Collaborative filtering cost function

The collaborative filtering cost function is given by
$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2
{
+\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
}$$
For all $i$, $j$ where $r(i,j)$ equals $1$ and could be written:

$$
= \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2
+\text{regularization}
$$



### 4.2 Implementation

In [55]:
def Cost_func(X, W, b, Y, R, lambda_):
    """
    Compute the cost function for collaborative filtering.

    Parameters:
    - X: Matrix of movie feature vectors (num_movies x num_features)
    - W: Matrix of user parameter vectors (num_users x num_features)
    - b: Vector of user bias terms (1 x num_users)
    - Y: Matrix of actual ratings (num_movies x num_users)
    - R: Binary-valued indicator matrix (num_movies x num_users)
    - lambda_: Regularization parameter

    Returns:
    - J: Scalar value representing the cost
    """
    nm, nu = Y.shape
    J = 0
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += r * np.square((np.dot(w,x) + b_j - y ))
    J += (lambda_) * (np.sum(np.square(W)) + np.sum(np.square(X)))
    J = J/2
    

    return J

**Vectorized Implementation**

Computing the cost $J$ using vectorized implementation will enhance efficiency, as the cost function will be called frequently to update the parameters it is recommended to have vectorized implementation of cost function


In [56]:
import tensorflow as tf

def Cost_func(X, W, b, Y, R, lambda_):
    """
    Compute the cost function for collaborative filtering.

    Parameters:
    - X: Matrix of movie feature vectors (num_movies x num_features)
    - W: Matrix of user parameter vectors (num_users x num_features)
    - b: Vector of user bias terms (1 x num_users)
    - Y: Matrix of actual ratings (num_movies x num_users)
    - R: Binary-valued indicator matrix (num_movies x num_users)
    - lambda_: Regularization parameter

    Returns:
    - J: Scalar value representing the cost
    """

    # Compute predicted ratings
    predictions = tf.matmul(X, tf.transpose(W)) + b

    # Compute error
    error = (predictions - Y) * R

    # Compute squared error
    squared_error = tf.reduce_sum(error ** 2)

    # Compute regularization term
    reg_term = (lambda_ / 2) * (tf.reduce_sum(W ** 2) + tf.reduce_sum(X ** 2))

    # Compute total cost
    J = 0.5 * squared_error + reg_term

    return J



## 5 - Learning movie recommendations 
------------------------------

Now we have to start training your algorithm to make movie recommendations for yourself. Starting we will Normalize The data set Y

In [57]:
Ymean = (np.sum(Y*R,axis=1)/(np.sum(R, axis=1)+1e-12)).reshape(-1,1)
Ynorm = Y - np.multiply(Ymean, R)

Let's Initialize the parameters w,x,b and select the Adam optimizer abd intialise the optimizer.

In [58]:
num_movies, num_users = Y.shape
num_features = 100

W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

optimizer = keras.optimizers.Adam(learning_rate=1e-1)

Let's now train the collaborative filtering model. To learn the parameters $\mathbf{X}$, $\mathbf{W}$, and $\mathbf{b}$. 

In the context of learning $w$, $b$, and $x$ simultaneously, the conventional workflow of TensorFlow's neural network package, which includes steps like `Model`, `Compile()`, `Fit()`, and `Predict()`, is not directly applicable. Instead, a custom training loop is needed.

Recalling the gradient descent algorithm:
- Iterate until convergence:
    - Compute the forward pass.
    - Compute the derivatives of the loss with respect to the parameters.
    - Update the parameters using the learning rate and the computed derivatives.

TensorFlow simplifies this process by automatically computing derivatives. Operations performed on TensorFlow Variables are tracked within a `tf.GradientTape()` context. Later, when `tape.gradient()` is called, it computes the gradient of the loss with respect to the tracked variables. These gradients can then be utilized to update the parameters using an optimizer.

In [59]:
iterations = 400
lambda_ = 1
for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = Cost_func(X, W, b, Ynorm, R, lambda_)
       
    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 2277677.1
Training loss at iteration 20: 133730.1
Training loss at iteration 40: 50353.5
Training loss at iteration 60: 23716.7
Training loss at iteration 80: 13069.6
Training loss at iteration 100: 8117.9
Training loss at iteration 120: 5554.5
Training loss at iteration 140: 4132.4
Training loss at iteration 160: 3304.7
Training loss at iteration 180: 2804.6
Training loss at iteration 200: 2492.2
Training loss at iteration 220: 2290.9
Training loss at iteration 240: 2157.3
Training loss at iteration 260: 2066.1
Training loss at iteration 280: 2001.9
Training loss at iteration 300: 1955.4
Training loss at iteration 320: 1920.9
Training loss at iteration 340: 1894.6
Training loss at iteration 360: 1874.1
Training loss at iteration 380: 1857.9


<a name="6"></a>
## 6 - Recommendations

Now we will take a new user with some ratings already given, and then we will learn parameters and then our algorithm will then make recommendations for us according to our preferences. 

In [60]:
movieList_df = pd.read_csv('./data/small_movie_list.csv', header=0, index_col=0,  delimiter=',', quotechar='"')


my_ratings = np.zeros(num_movies)          


my_ratings[2700] = 5 # Toy Story 3 (2010)
my_ratings[2609] = 2 # Persuasion (2007)
my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)

my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

print('\nNew user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {movieList_df.loc[i,"title"]}');


New user ratings:

Rated 5.0 for  Shrek (2001)
Rated 5.0 for  Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Rated 2.0 for  Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Rated 5.0 for  Harry Potter and the Chamber of Secrets (2002)
Rated 5.0 for  Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Rated 5.0 for  Lord of the Rings: The Return of the King, The (2003)
Rated 3.0 for  Eternal Sunshine of the Spotless Mind (2004)
Rated 5.0 for  Incredibles, The (2004)
Rated 2.0 for  Persuasion (2007)
Rated 5.0 for  Toy Story 3 (2010)
Rated 3.0 for  Inception (2010)
Rated 1.0 for  Louis Theroux: Law & Disorder (2008)
Rated 1.0 for  Nothing to Declare (Rien à déclarer) (2010)


In [61]:
Y    = np.c_[my_ratings, Y]
R    = np.c_[(my_ratings != 0).astype(int), R]
Ymean = (np.sum(Y*R,axis=1)/(np.sum(R, axis=1)+1e-12)).reshape(-1,1)
Ynorm = Y - np.multiply(Ymean, R)
num_movies, num_users = Y.shape
num_features = 100


W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')


optimizer = keras.optimizers.Adam(learning_rate=1e-1)
iterations = 200
lambda_ = 1
for iter in range(iterations):
     
    with tf.GradientTape() as tape:
        cost_value = Cost_func(X, W, b, Ynorm, R, lambda_)

    grads = tape.gradient( cost_value, [X,W,b] )
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

**Prediction** :
1. Computes predictions for movie ratings using collaborative filtering by multiplying movie feature vectors with user parameter vectors, adding user bias terms, and restoring the mean rating.

2. Sorts the predicted ratings in descending order to identify top recommendations for movies that the user has not rated yet.

3. Prints out the top recommended movies along with their predicted ratings.

4. Compares the predicted ratings with the original ratings provided by the user for movies they have rated, offering a comparison between actual and predicted preferences.

In [62]:
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restore the mean
pm = p + Ymean

my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movieList[j]}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movieList[i]}')

Predicting rating 4.91 for movie 'Salem's Lot (2004)
Predicting rating 4.90 for movie Particle Fever (2013)
Predicting rating 4.90 for movie Eva (2011)
Predicting rating 4.89 for movie Nine Lives of Tomas Katz, The (2000)
Predicting rating 4.89 for movie Colourful (Karafuru) (2010)
Predicting rating 4.89 for movie Eichmann (2007)
Predicting rating 4.89 for movie I'm the One That I Want (2000)
Predicting rating 4.88 for movie Paper Birds (Pájaros de papel) (2010)
Predicting rating 4.88 for movie Indignation (2016)
Predicting rating 4.88 for movie Won't You Be My Neighbor? (2018)
Predicting rating 4.88 for movie I Am Not Your Negro (2017)
Predicting rating 4.88 for movie Act of Killing, The (2012)


Original vs Predicted ratings:

Original 5.0, Predicted 4.89 for Shrek (2001)
Original 5.0, Predicted 4.89 for Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Original 2.0, Predicted 2.07 for Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
O

Displaying Top movies with highest average rating and also watched by minimum of 20 users

In [63]:
filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_predictions
movieList_df = movieList_df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)

,pred,mean rating,number of ratings,title
2079,4.687964,4.158537,41,In Bruges (2008)
2395,4.363327,4.136364,88,Inglourious Basterds (2009)
929,4.894457,4.118919,185,"Lord of the Rings: The Return of the King, The..."
2700,4.856552,4.109091,55,Toy Story 3 (2010)
393,4.529161,4.106061,198,"Lord of the Rings: The Fellowship of the Ring,..."
1598,4.364169,4.027778,36,Thank You for Smoking (2006)
2804,4.436876,3.989362,47,Harry Potter and the Deathly Hallows: Part 1 (...
785,4.385637,3.974138,58,28 Days Later (2002)
877,4.603886,3.961832,131,Kill Bill: Vol. 1 (2003)
1431,4.376377,3.940000,50,Serenity (2005)
